In [1]:
import numpy as np
import pandas as pd
import zipfile

from sklearn.metrics import accuracy_score

In [2]:
df_train = pd.read_csv('input/train.csv')

df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
display('training data shape')
display(df_train.shape)

def print_percent(labels):
    for label in labels:
        display('{label} percentage {percent}'.format(label=label, percent = len(df_train.loc[df_train[label] == 1])/df_train.shape[0])) 

'training data shape'

(159571, 8)

In [4]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

print_percent(classes)

y_true = df_train[classes].as_matrix()

y_pred = np.zeros(y_true.shape)


'toxic percentage 0.09584448302009764'

'severe_toxic percentage 0.009995550569965721'

'obscene percentage 0.052948217407925'

'threat percentage 0.002995531769557125'

'insult percentage 0.04936360616904074'

'identity_hate percentage 0.00880485802558109'

In [5]:
accuracy_score(y_true, y_pred)

0.89832112351241766

In [23]:
df_severe_toxic = df_train.loc[(df_train['severe_toxic'] == 1)]
df_severe_toxic_not_toxic =df_train.loc[(df_train['severe_toxic'] == 1) & (df_train['toxic'] == 0)]
df_severe_toxic_and_toxic =df_train.loc[(df_train['severe_toxic'] == 1) & (df_train['toxic'] == 1)]

df_severe_toxic_not_insult = df_train.loc[(df_train['severe_toxic'] == 1) & (df_train['insult'] == 0)]

print(df_severe_toxic.shape[0])
print(df_severe_toxic_not_toxic.shape[0] / df_severe_toxic.shape[0])
print(df_severe_toxic_and_toxic.shape[0] / df_severe_toxic.shape[0])

print(df_severe_toxic_not_insult.shape[0] / df_severe_toxic.shape[0])



1595
0.0
1.0
0.14043887147335424


In [37]:
corr = df_train.corr()

print(~np.tril(np.ones(corr.shape)).astype(np.bool))

corr_tril = corr.where(~np.tril(np.ones(corr.shape)).astype(np.bool))

corr_tril = corr_tril.stack()

print(corr_tril[corr_tril > 0.5])

[[False  True  True  True  True  True]
 [False False  True  True  True  True]
 [False False False  True  True  True]
 [False False False False  True  True]
 [False False False False False  True]
 [False False False False False False]]
toxic    obscene    0.676515
         insult     0.647518
obscene  insult     0.741272
dtype: float64


In [45]:
df_ones = df_train.loc[(df_train.toxic == 1) | (df_train.severe_toxic == 1) | (df_train.obscene == 1) | (df_train.insult == 1) | (df_train.identity_hate == 1) | (df_train.threat == 1)]

#display(df_ones)

ones_corr = df_ones.corr()

ones_corr.where(~np.tril(np.ones(ones_corr.shape)).astype(np.bool))

,toxic,severe_toxic,obscene,threat,insult,identity_hate
toxic,NaN,0.081465,-0.020260,-0.002464,-0.042957,-0.021089
severe_toxic,NaN,NaN,0.284441,0.079589,0.247133,0.128725
obscene,NaN,NaN,NaN,0.038003,0.506815,0.131760
threat,NaN,NaN,NaN,NaN,0.054652,0.073364
insult,NaN,NaN,NaN,NaN,NaN,0.209547
identity_hate,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_test = pd.read_csv('input/test.csv')

display(df_test.head())

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [7]:
for c in classes:
    df_test[c]= np.zeros((df_test.shape[0], 1))
    
df_test = df_test.drop(labels=['comment_text'], axis=1)

In [8]:
display(df_test.head())

df_test.to_csv('output/baseline_submission.csv', index=False)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.0,0.0,0.0,0.0,0.0,0.0
1,0000247867823ef7,0.0,0.0,0.0,0.0,0.0,0.0
2,00013b17ad220c46,0.0,0.0,0.0,0.0,0.0,0.0
3,00017563c3f7919a,0.0,0.0,0.0,0.0,0.0,0.0
4,00017695ad8997eb,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
display(df_test.shape)

zipfile.ZipFile(file='output/baseline_submission.csv.zip', mode='w').write('output/baseline_submission.csv', arcname='baseline_submission.csv')

(153164, 7)

In [23]:
c_count = [len(df_train[df_train[c] == 1]) for c in classes]

In [42]:
m = df_train.shape[0]

import math as math

weights = [max(1.0, math.log(0.28 * m / float(c))) for c in c_count]
# math.log(mu*total/float(labels_dict[key]))

In [43]:
print(c_count)
print(weights)

[15294, 1595, 8449, 478, 7877, 1405]
[1.0720626938099167, 3.3326495521951425, 1.6654751972612425, 4.537667834922631, 1.735576167710763, 3.459485985646241]
